In [3]:
from datetime import datetime, date, time, timedelta
from itertools import islice
from geopy.distance import great_circle
from operator import itemgetter
import matplotlib.pylab as plt
from pandas import DataFrame
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.express as px
import pandas as pd
import numpy
import json
import math


DELTA_GNSS = 0.05 #equivalent to about 5-6 km
DELTA_GNSS_INIT = 1 #equivalent to about 5-6 km
DELTA_POS = 0.0001 #equivalent to about 100 m

#six decimal are enough for a decimeter precision
#0.1 = 11.1 km
#0.01 = 1.11 km
#...
#0.000001 = 0.111 m
uav_latitude = 45.0059289
uav_longitude = 10.461025
uav_coo = (uav_latitude, uav_longitude)

#support variables
bts_latitude = 0
bts_longitude = 0

#Load BTS database
BTS_DATA = pd.read_csv('bts_windtre.csv', sep=';')
BTS_DATA = BTS_DATA.drop(columns=['tech','mcc','mnc','lac_tac','cid','psc_pci','arfcn','azimuth','height','tilt_mech','tilt_el'])
BTS_DATA = BTS_DATA.round(5)

BTS_DATA = BTS_DATA.drop_duplicates(subset=['site_name'])
BTS_DATA = BTS_DATA.drop_duplicates(subset=['cell_lat'])
BTS_DATA = BTS_DATA.drop_duplicates(subset=['cell_long'])

###In order to limit processing, limit the DF to cells in 100km radius
Nearby_Cells_Lat = abs(BTS_DATA['cell_lat']-uav_coo[0])<=DELTA_GNSS_INIT
Nearby_Cells_Lon = abs(BTS_DATA['cell_long']-uav_coo[1])<=DELTA_GNSS_INIT

BTS_DATA = BTS_DATA[Nearby_Cells_Lon]
BTS_DATA = BTS_DATA[Nearby_Cells_Lat]
###Must be deleted in the final version

<ipython-input-3-8d9d0b6f8fe6>:47: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  BTS_DATA = BTS_DATA[Nearby_Cells_Lat]


In [18]:
#Bearing Angle
def calculate_bearing(pointA, pointB):
    """
    Calculates the bearing between two points.
    The formulae used is the following:
        θ = atan2(sin(Δlong).cos(lat2),
                  cos(lat1).sin(lat2) − sin(lat1).cos(lat2).cos(Δlong))
    :Parameters:
      - `pointA: The tuple representing the latitude/longitude for the
        first point. Latitude and longitude must be in decimal degrees
      - `pointB: The tuple representing the latitude/longitude for the
        second point. Latitude and longitude must be in decimal degrees
    :Returns:
      The bearing in degrees
    :Returns Type:
      float
    """
    if (type(pointA) != tuple) or (type(pointB) != tuple):
        raise TypeError("Only tuples are supported as arguments")

    lat1 = math.radians(pointA[0])
    lat2 = math.radians(pointB[0])

    diffLong = math.radians(pointB[1] - pointA[1])

    x = math.sin(diffLong) * math.cos(lat2)
    y = math.cos(lat1) * math.sin(lat2) - (math.sin(lat1)
            * math.cos(lat2) * math.cos(diffLong))

    initial_bearing = math.atan2(x, y)

    # Now we have the initial bearing but math.atan2 return values
    # from -180° to + 180° which is not what we want for a compass bearing
    # The solution is to normalize the initial bearing as shown below
    initial_bearing = math.degrees(initial_bearing)
    compass_bearing = (initial_bearing + 360) % 360

    return compass_bearing

#Find nearby cells
def CellFind(uav_coo):

    Nearby_Cells_Lat = abs(BTS_DATA['cell_lat']-uav_coo[0])<=DELTA_GNSS
    Nearby_Cells_Lon = abs(BTS_DATA['cell_long']-uav_coo[1])<=DELTA_GNSS

    NEARBY_BTS = BTS_DATA[Nearby_Cells_Lon]
    NEARBY_BTS = NEARBY_BTS[Nearby_Cells_Lat]

    #Compute distance and bearing of nearby cells
    nearby_bts_distances = []
    nearby_bts_bearing = []

    for i in range(len(NEARBY_BTS)):
        cell = NEARBY_BTS.iloc[i]
        bts_coo = (cell.cell_lat, cell.cell_long)
        nearby_bts_distances.append(great_circle(uav_coo, bts_coo).m)
        nearby_bts_bearing.append(calculate_bearing(uav_coo, bts_coo))


    NEARBY_BTS['distance'] = nearby_bts_distances
    NEARBY_BTS['bearing'] = nearby_bts_bearing

    NEARBY_BTS = NEARBY_BTS.sort_values(by='distance')

    return(NEARBY_BTS)

In [19]:
#Import UAV Log network Ping data from JSON file
data = []
with open('UAV_LOG.json') as f:
    for line in f:
        try:
            data.append(json.loads(line))
        except:
            print('Skip line')

#Convert JSON data from Python dict to DF
UAV_LOG = pd.json_normalize(data)

#Drop unusfull data
UAV_LOG = UAV_LOG.drop(columns=['uav_param.status','uav_param.mode','interface','gps_param.groundspeed','gps_param.velocity.vx','uav_param.attitude.pitch','uav_param.attitude.yaw','uav_param.attitude.roll','gps_param.velocity.vy','gps_param.velocity.vz','gps_param.hdop','gps_param.vdop','gps_param.fix_type'])

#Remove duplicate entries
UAV_LOG = UAV_LOG.drop_duplicates('timestamp')
UAV_LOG = UAV_LOG.apply(pd.to_numeric, errors='ignore')

#Remove false flights
UAV_bench = UAV_LOG['uav_param.battery']>=10.0
UAV_LOG = UAV_LOG[UAV_bench]

#Remove no GPS fix data
GPS_Null = UAV_LOG['gps_param.satellites']>=6
UAV_LOG = UAV_LOG[GPS_Null]

#Sort by time for time clustering
UAV_LOG = UAV_LOG.sort_values(by='timestamp')


Skip line
Skip line
Skip line
Skip line
Skip line
Skip line
Skip line
Skip line


In [20]:
#Time Clustering Function
#Convert timestamp of dataframe to python datetime
UAV_LOG.timestamp = pd.to_datetime(UAV_LOG["timestamp"])
UAV_LOG.timestamp = UAV_LOG.timestamp.dt.to_pydatetime()

#CLUSTER LTE DF
#Define the minimum time difference between flights, in order to identy clusters of data
delta = timedelta(minutes=2)
final = list()
tmp = list()

#First row
tmp.append(UAV_LOG.iloc[0,:])

#First datetime
previous = UAV_LOG.iloc[0,0]

#Scan UAV Log dataframe looking for clusters and then append them into a list of dataframes
for index, row in islice(UAV_LOG.iterrows(), 0, None):
    dt = row.timestamp
    if dt - previous > delta:
        final.append(tmp)
        tmp = list()
    tmp.append(row)
    previous = dt

#Append last cluster to the list of DF
final.append(tmp)

#Remove flight with less than 100 rows
flights = []
for i in range(len(final)):
    if len(final[i])>1000:
        flights.append(final[i])

#Print N° of flights
print("Found ", len(flights), " flights with data")

#Print flight lenght
for i in range(len(flights)):
    print("Flight N°", i+1, "has ", len(flights[i]), " rows of data")

print('')

Found  11  flights with data
Flight N° 1 has  1029  rows of data
Flight N° 2 has  1807  rows of data
Flight N° 3 has  2755  rows of data
Flight N° 4 has  2013  rows of data
Flight N° 5 has  1880  rows of data
Flight N° 6 has  5348  rows of data
Flight N° 7 has  6511  rows of data
Flight N° 8 has  5091  rows of data
Flight N° 9 has  4600  rows of data
Flight N° 10 has  4760  rows of data
Flight N° 11 has  4363  rows of data



In [21]:
DF = DataFrame (flights[6])

for log in range(len(DF)):
    uav_coo = (DF.iloc[log]['gps_param.latitude'], DF.iloc[log]['gps_param.longitude'])
    BTS = CellFind(uav_coo)
    #print(BTS.iloc[0])
    #print(DF.iloc[log])

DF

<ipython-input-18-8374e3e56ef3>:47: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  NEARBY_BTS = NEARBY_BTS[Nearby_Cells_Lat]


,timestamp,uav_param.battery,gps_param.latitude,gps_param.longitude,gps_param.altitude,gps_param.head,gps_param.satellites
46140,2021-05-26 18:11:01.334484,25.042,45.005928,10.461025,12.12,78,16
46141,2021-05-26 18:11:01.546270,25.042,45.005928,10.461025,12.12,78,16
46142,2021-05-26 18:11:01.747840,25.042,45.005928,10.461025,12.12,78,16
46143,2021-05-26 18:11:01.949503,25.042,45.005928,10.461025,12.12,78,16
46144,2021-05-26 18:11:02.151131,25.042,45.005928,10.461025,12.12,78,16
...,...,...,...,...,...,...,...
52646,2021-05-26 18:33:15.878921,22.017,45.005931,10.461046,13.03,76,15
52647,2021-05-26 18:33:16.081513,21.984,45.005931,10.461046,13.04,76,15
52648,2021-05-26 18:33:16.283872,21.984,45.005931,10.461046,13.04,76,15
52649,2021-05-26 18:33:16.486417,21.984,45.005931,10.461046,13.04,76,15


In [22]:
DATA = []

for f in range(len(DF)):
    coo = (DF.iloc[f]['gps_param.latitude'], DF.iloc[f]['gps_param.longitude'])
    DATA.append(coo)

In [23]:
BTS

,node_id,band,site_name,cell_lat,cell_long,cell_name,distance,bearing
54806,124049,1800,Sabbioneta Via San Remigio,45.00647,10.49191,B3 S1 DSS Sabbioneta Via San Remigio,2427.214360,88.574688
32764,116359,1800,Casalmaggiore Via Marconi,44.99652,10.42122,B3 S1 DSS Casalmaggiore Via Marconi,3301.561374,251.534606
31518,116049,1800,Casalmaggiore Via Vanoni,44.98297,10.43259,B3 S1 DSS Casalmaggiore Via Vanoni,3394.942835,221.241644
32146,116093,1800,Rivarolo del Re ed Uniti Via Belfiore,45.03348,10.48455,B3 S1 DSS Rivarolo del Re ed Uniti Via Belfiore,3577.243432,31.085012
31764,116061,1800,Casalmaggiore,44.97056,10.46489,B3 S1 DSS Casalmaggiore,3944.704151,175.603455
32788,116360,1800,Casalmaggiore Via Saffi,44.98517,10.41484,B3 S1 DSS Casalmaggiore Via Saffi,4304.686653,237.585168


In [24]:
from ipyleaflet import Map, basemaps, Marker, Icon
from ipywidgets import Layout

zoom = 25
DF = DataFrame (flights[3])

#Remove log for same points
#DF = DF.round(5)
DF = DF.drop_duplicates('gps_param.latitude')
DF = DF.drop_duplicates('gps_param.longitude')

m = Map(basemap=basemaps.OpenStreetMap.Mapnik, center=uav_coo, zoom=zoom, layout=Layout(width='100%', height='800px'))

#Define icons
icon_drone_front = Icon(icon_url='https://raw.githubusercontent.com/palia95/Antenna_Algo/main/icons/drone_front.png', icon_anchor=[16, 16], icon_size=[44, 44], rotate=True)
icon_drone_back = Icon(icon_url='https://raw.githubusercontent.com/palia95/Antenna_Algo/main/icons/drone_back.png', icon_anchor=[16, 16], icon_size=[44, 44], rotate=True)
icon_drone_left = Icon(icon_url='https://raw.githubusercontent.com/palia95/Antenna_Algo/main/icons/drone_left.png', icon_anchor=[16, 16], icon_size=[44, 44], rotate=True)
icon_drone_right = Icon(icon_url='https://raw.githubusercontent.com/palia95/Antenna_Algo/main/icons/drone_right.png', icon_anchor=[16, 16], icon_size=[44, 44], rotate=True)
icon_bts = Icon(icon_url='http://simpleicon.com/wp-content/uploads/antenna-2-64x64.png', icon_anchor=[32, 32], icon_size=[32, 32])

#Marker creation funcion
def MarkerDrone(uav_coo, icon_drone, uav_head):
    m.add_layer(Marker(location=uav_coo, icon=icon_drone, draggable=False, rotation_angle=uav_head))

def MarkerBTS(bts_coo):
    marker_bts = Marker(location=bts_coo, icon=icon_bts, draggable=False) 
    m.add_layer(marker_bts)

for bts in range(len(BTS)):
    bts_coo = BTS.iloc[bts].cell_lat, BTS.iloc[bts].cell_long
    MarkerBTS(bts_coo)

i = 0
prev_head = 0

while i < len(DF):
    uav_coo = (DF.iloc[i]['gps_param.latitude'], DF.iloc[i]['gps_param.longitude'])

    uav_head = float(DF.iloc[i]['gps_param.head'])

    bts_bearing = BTS.iloc[0].bearing

    if (uav_head!=prev_head):
        print("UAV: "+str(uav_coo)+' '+str(uav_head)+' '+str(bts_bearing))
        if (math.cos(math.radians(uav_head-45)))<math.cos(math.radians(bts_bearing))<=(math.cos(math.radians(uav_head+45))):
            icon_drone = icon_drone_front
            MarkerDrone(uav_coo, icon_drone, uav_head)
            print('Front')
        elif (math.sin(math.radians(uav_head+45)))<math.sin(math.radians(bts_bearing))<=(math.sin(math.radians(uav_head+90+45))):
            icon_drone = icon_drone_right
            MarkerDrone(uav_coo, icon_drone, uav_head)
            print('Right')
        elif (math.cos(math.radians(uav_head+180-45)))<math.cos(math.radians(bts_bearing))<=(math.cos(math.radians(uav_head+180+45))):
            icon_drone = icon_drone_back
            MarkerDrone(uav_coo, icon_drone, uav_head)
            print('Back')
        elif (math.sin(math.radians(uav_head+270-45)))<math.sin(math.radians(bts_bearing))<=(math.sin(math.radians(uav_head+270+45))):
            icon_drone = icon_drone_left
            MarkerDrone(uav_coo, icon_drone, uav_head)
            print('Left')

    prev_head = uav_head
    i += 3

m

UAV: (45.0059388, 10.4609999) 76.0 88.57468830308204
Back
UAV: (45.0059346, 10.4609987) 74.0 88.57468830308204
Back
UAV: (45.0059357, 10.4609964) 77.0 88.57468830308204
Back
UAV: (45.0059529, 10.4610154) 34.0 88.57468830308204
UAV: (45.0059825, 10.4610546) 37.0 88.57468830308204
UAV: (45.0060225, 10.4611011) 32.0 88.57468830308204
UAV: (45.0060688, 10.4611479) 35.0 88.57468830308204
UAV: (45.0061168, 10.4611956) 34.0 88.57468830308204
UAV: (45.0061662, 10.4612454) 36.0 88.57468830308204
UAV: (45.0062154, 10.4612983) 48.0 88.57468830308204
Back
UAV: (45.006252, 10.4613663) 59.0 88.57468830308204
Back
UAV: (45.0062792, 10.4614439) 63.0 88.57468830308204
Back
UAV: (45.006305, 10.461527) 66.0 88.57468830308204
Back
UAV: (45.0063564, 10.4617001) 68.0 88.57468830308204
Back
UAV: (45.0063818, 10.4617875) 69.0 88.57468830308204
Back
UAV: (45.0064525, 10.4620525) 68.0 88.57468830308204
Back
UAV: (45.0064773, 10.4621417) 69.0 88.57468830308204
Back
UAV: (45.0065048, 10.4622438) 68.0 88.574688303

Map(center=[45.0059312, 10.461046], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title'…

In [25]:
print(math.cos(math.radians(uav_head))-math.cos(math.radians(45)))
print(math.cos(math.radians(uav_head-45)))

-0.7071067811865478
-0.7071067811865477


In [12]:
    if (uav_head!=prev_head):
        print("UAV: "+str(uav_coo)+' '+str(uav_head)+' '+str(bts_bearing))
        if (math.cos(uav_head)-math.cos(45))<math.cos(bts_bearing)<=(math.cos(uav_head)+math.cos(45)):
            print('First case')
            if 360+(uav_head-45) < bts_bearing <= 360:
                icon_drone = icon_drone_front
                MarkerDrone(i, uav_coo, icon_drone, uav_head)
                print('Front')
            elif 0 < bts_bearing <= uav_head+45:
                icon_drone = icon_drone_front
                MarkerDrone(i, uav_coo, icon_drone, uav_head)
                print('Front')
            elif uav_head+45 < bts_bearing <= uav_head+45+90:
                icon_drone = icon_drone_right
                MarkerDrone(i, uav_coo, icon_drone, uav_head)
                print('Right')
            elif uav_head+45+90 < bts_bearing <= uav_head+45+90+90:
                icon_drone = icon_drone_back
                MarkerDrone(i, uav_coo, icon_drone, uav_head)
                print('Back')
            elif uav_head+45+90+90 < bts_bearing <= uav_head+45+90+90+90:
                icon_drone = icon_drone_left
                MarkerDrone(i, uav_coo, icon_drone, uav_head)
                print('Left')

        elif 315 < uav_head <= 360:
            print('Second case')
            if uav_head-45 < bts_bearing <= 360:
                icon_drone = icon_drone_front
                MarkerDrone(i, uav_coo, icon_drone, uav_head)
                print('Front')
            elif 0 < bts_bearing <= (uav_head+45)-360:
                icon_drone = icon_drone_front
                MarkerDrone(i, uav_coo, icon_drone, uav_head)
                print('Front')
            elif (uav_head+45)-360 < bts_bearing <= (uav_head+45+90)-360:
                icon_drone = icon_drone_right
                MarkerDrone(i, uav_coo, icon_drone, uav_head)
                print('Right')
            elif (uav_head+45+90)-360 < bts_bearing <= (uav_head+45+90+90)-360:
                icon_drone = icon_drone_back
                MarkerDrone(i, uav_coo, icon_drone, uav_head)
                print('Back')
            elif (uav_head+45+90+90)-360 < bts_bearing <= uav_head-45:
                icon_drone = icon_drone_left
                MarkerDrone(i, uav_coo, icon_drone, uav_head)
                print('Left')

        else:
            print('Third case')
            if uav_head-45 < bts_bearing <= uav_head+45:
                icon_drone = icon_drone_front
                MarkerDrone(i, uav_coo, icon_drone, uav_head)
                print('Front')
            elif uav_head+45 < bts_bearing <= uav_head+45+90:
                icon_drone = icon_drone_right
                MarkerDrone(i, uav_coo, icon_drone, uav_head)
                print('Right')
            elif uav_head+45+90 < bts_bearing <= uav_head+45+90+90:
                icon_drone = icon_drone_back
                MarkerDrone(i, uav_coo, icon_drone, uav_head)
                print('Back')
            elif uav_head+45+90+90 < bts_bearing <= 360:
                icon_drone = icon_drone_left
                MarkerDrone(i, uav_coo, icon_drone, uav_head)
                print('Left')
            elif 0 < bts_bearing <= uav_head-45:
                icon_drone = icon_drone_left
                MarkerDrone(i, uav_coo, icon_drone, uav_head)
                print('Left')

In [13]:
math.cos(math.radians(30))

0.8660254037844387

In [14]:
DF

,timestamp,uav_param.battery,gps_param.latitude,gps_param.longitude,gps_param.altitude,gps_param.head,gps_param.satellites
34332,2021-05-26 12:07:28.149208,24.861,45.005939,10.461000,11.95,76,16
34333,2021-05-26 12:07:28.650743,24.857,45.005938,10.460999,11.91,76,16
34336,2021-05-26 12:07:30.158731,24.858,45.005938,10.460999,11.89,76,16
34338,2021-05-26 12:07:31.163135,24.865,45.005938,10.460999,11.92,76,16
34339,2021-05-26 12:07:31.667127,24.861,45.005937,10.460999,11.94,76,16
...,...,...,...,...,...,...,...
36276,2021-05-26 12:23:45.657753,21.276,45.005940,10.460999,18.51,270,16
36282,2021-05-26 12:23:48.676832,21.294,45.005940,10.461003,17.06,271,16
36284,2021-05-26 12:23:49.682206,21.274,45.005939,10.461004,16.48,271,16
36318,2021-05-26 12:24:06.786023,21.947,45.005941,10.461005,12.22,269,17


In [15]:
import dash_leaflet as dl
from dash import Dash, html

#Drone path
patterns = [dict(repeat='10', dash=dict(pixelSize=5, pathOptions=dict(color='#000', weight=1, opacity=0.2))), dict(offset='16%', repeat='5%')]
drone_path = dl.PolylineDecorator(positions=DATA, patterns=patterns)

#Rotated custom marker
iconUrl = "https://github.com/palia95/Antenna_Algo/blob/main/icons/drone_front.png"
marker = dict(rotate=True, markerOptions=dict(icon=dict(iconUrl=iconUrl, iconAnchor=[16, 16])))

#marker_drone = dl.Marker(position=(DF.iloc[0]['gps_param.latitude'], DF.iloc[0]['gps_param.longitude']), icon=dict(iconUrl=iconUrl), draggable=False, rotationAngle=float(DF.iloc[0]['gps_param.head'])) 
#marker_bts = Marker(location=(BTS.iloc[0].cell_lat, BTS.iloc[0].cell_long), icon=icon_bts, draggable=False) 
#m.add_layer(marker_bts)
#dl.Marker(position=(DF.iloc[0]['gps_param.latitude'], DF.iloc[0]['gps_param.longitude']))
#m.add_layer(marker_drone)

iconUrl = "https://dash-leaflet.herokuapp.com/assets/icon_plane.png"
marker = dict(rotate=True, markerOptions=dict(icon=dict(iconUrl=iconUrl, iconAnchor=[16, 16])))

#Create app
app = Dash()
app.layout = html.Div(dl.Map([dl.TileLayer(), drone_path, dl.Marker(position=(DF.iloc[0]['gps_param.latitude'], DF.iloc[0]['gps_param.longitude']), marker = marker)], zoom=15, center=uav_coo), style={'width': '100%', 'height': '150vh', 'margin': "auto", "display": "block"})

if __name__ == '__main__':
    app.run_server()

TypeError: The `dash_leaflet.Marker` component (version 0.1.23) received an unexpected keyword argument: `marker`
Allowed arguments: alt, attribution, autoPan, autoPanPadding, autoPanSpeed, bubblingMouseEvents, children, draggable, icon, id, interactive, keyboard, n_clicks, opacity, pane, position, riseOffset, riseOnHover, title, zIndexOffset

In [ ]:
DF

,timestamp,uav_param.battery,gps_param.latitude,gps_param.longitude,gps_param.altitude,gps_param.head,gps_param.satellites
46140,2021-05-26 18:11:01.334484,25.042,45.005928,10.461025,12.12,78,16
46141,2021-05-26 18:11:01.546270,25.042,45.005928,10.461025,12.12,78,16
46142,2021-05-26 18:11:01.747840,25.042,45.005928,10.461025,12.12,78,16
46143,2021-05-26 18:11:01.949503,25.042,45.005928,10.461025,12.12,78,16
46144,2021-05-26 18:11:02.151131,25.042,45.005928,10.461025,12.12,78,16
...,...,...,...,...,...,...,...
52646,2021-05-26 18:33:15.878921,22.017,45.005931,10.461046,13.03,76,15
52647,2021-05-26 18:33:16.081513,21.984,45.005931,10.461046,13.04,76,15
52648,2021-05-26 18:33:16.283872,21.984,45.005931,10.461046,13.04,76,15
52649,2021-05-26 18:33:16.486417,21.984,45.005931,10.461046,13.04,76,15
